In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta, date
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.patches as patches
from matplotlib.colors import TwoSlopeNorm
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from mapping_tickers import *
from mapping_portfolio_downloads import *
from mapping_plot_attributes import theme_style
from utils import *
from mapping_portfolio_downloads import *
from download_data import DownloadData
from analyze_prices import AnalyzePrices

In [2]:
tickers = list(magnificent_7_tickers.keys())
end_date = datetime.today()
hist_years, hist_months, hist_days = 1, 0, 0
start_date = datetime(end_date.year - hist_years, end_date.month - hist_months, end_date.day - hist_days)
tk_market = '^GSPC'

hist_data = DownloadData(end_date, start_date, tickers, tk_market)

downloaded_data = hist_data.download_yh_data(start_date, end_date, tickers, tk_market)
df_adj_close = downloaded_data['Adj Close']
df_close = downloaded_data['Close']
dict_ohlc = downloaded_data['OHLC']

tk = 'AAPL'
df_ohlc = dict_ohlc[tk]
ohlc_tk = df_ohlc.copy()
adj_close_tk = df_adj_close[tk]
close_tk = df_close[tk]
open_tk = ohlc_tk['Open']
high_tk = ohlc_tk['High']
low_tk = ohlc_tk['Low']

price_type_map = {
    'Adj Close': adj_close_tk,
    'Adjusted Close': adj_close_tk,
    'Close': close_tk,
    'Open': open_tk,
    'High': high_tk,
    'Low': low_tk
}

# display(df_adj_close)
# display(df_close)
# display(df_ohlc)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

The portfolio data will be truncated to end at the latest available date of 2024-09-20.


In [3]:
# From https://github.com/matplotlib/mplfinance/blob/master/examples/indicators/rsi.py

def relative_strength(prices, period = 14):
    """
    http://stockcharts.com/school/doku.php?id=chart_school:glossary_r#relativestrengthindex
    http://www.investopedia.com/terms/r/rsi.asp
    """
    
    deltas = np.diff(prices)
    seed = deltas[:period + 1]
    up = seed[seed >= 0].sum() / period
    down = -seed[seed < 0].sum() / period
    rs = up / down
    array_rsi = np.zeros_like(prices)
    array_rsi[:period] = 100. - 100. / (1. + rs)

    for i in range(period, len(prices)):
        delta = deltas[i - 1]  # cause the diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up * (period - 1) + upval) / period
        down = (down * (period - 1) + downval) / period

        rs = up / down
        array_rsi[i] = 100. - 100. / (1. + rs)

    rsi = pd.Series(data = array_rsi, index = prices.index.astype(str))
    rsi_type = f'{period}'

    rsi_data = {
        'rsi': rsi,
        'type': rsi_type
    }

    return rsi_data

In [4]:
def plot_rsi_hlines_plotly(
    rsi_data,
    tk,
    oversold_threshold = 30,
    overbought_threshold = 70,
    n_ticks_max = 48,
    plot_width = 1450,
    plot_height = 750,
    title_font_size = 32,
    theme = 'dark',
    overlay_price = False,
    df_price = None,
    price_type = 'adjusted close'
):
    """
    rsi_data:   output from relative_strength()
    tk:         ticker for which to plot RSI
    price_type: normally 'adjusted close' or 'close', whatever the RSI is based on
    df_price:   dataframe/series of prices to overlay (if overlay_price is True)

    """

    rsi = rsi_data['rsi']
    rsi_type = rsi_data['type']
    
    style = theme_style[theme]

    title_rsi = f'{tk} Relative Strength Index {rsi_type} (%)'
    price_types = ['adjusted close', 'adj close', 'close', 'open', 'high', 'low']
    
    if price_type in price_types:
        price_name = 'Adjusted Close' if price_type == 'adj close' else price_type.title()
    else:
        price_name = 'Adjusted Close'

    if overlay_price:
        fig_rsi = make_subplots(specs=[[{'secondary_y': True}]])
    else:
        fig_rsi = make_subplots(rows = 1, cols = 1)

    x_min = rsi.index.min()
    x_max = rsi.index.max()
    
    rsi_hlines = pd.DataFrame(
        {
            'oversold': oversold_threshold,
            'overbought': overbought_threshold,
            '100': 100
        },
        index = rsi.index
    )

    # For some reason, the price overlay trace shows first in the legend if it's added last
    if overlay_price:
        fig_rsi.add_trace(
            go.Scatter(
                x = rsi.index,
                y = df_price,
                line_color = style['basecolor'],
                name = price_name
            ),
            secondary_y = True
        )
    fig_rsi.add_trace(
        go.Scatter(
            x = rsi_hlines.index,
            y = rsi_hlines['oversold'],
            line_color = style['oversold_linecolor'],
            line_width = 2,
            fill = 'tozeroy',
            fillcolor = style['oversold_fillcolor'],
            name = f'Oversold < {oversold_threshold}%'
        ),
        secondary_y = False
    )
    fig_rsi.add_trace(
        go.Scatter(
            x = rsi_hlines.index,
            y = rsi_hlines['100'],
            line_color = 'black',
            line_width = 0,
            showlegend = False
        ),
        secondary_y = False
    )
    fig_rsi.add_trace(
        go.Scatter(
            x = rsi_hlines.index,
            y = rsi_hlines['overbought'],
            line_color = style['overbought_linecolor'],
            line_width = 2,
            fill = 'tonexty',  # fill to previous scatter trace
            fillcolor = style['overbought_fillcolor'],
            name = f'Overbought > {overbought_threshold}%'
        ),
        secondary_y = False
    )
    fig_rsi.add_trace(
        go.Scatter(
            x = rsi.index,
            y = rsi,
            line_color = style['rsi_linecolor'],
            line_width = 2,        
            name = f'RSI {rsi_type} (%)'
        ),
        secondary_y = False
    )

    # Add plot border
    fig_rsi.add_shape(
        type = 'rect',
        xref = 'x',  # use 'x' because of seconday axis - 'paper' does not work correctly
        yref = 'paper',
        x0 = x_min,
        x1 = x_max,
        y0 = 0,
        y1 = 1,
        line_color = style['x_linecolor'],
        line_width = 0.3
    )
    
    # Update layout and axes
    fig_rsi.update_layout(
        width = plot_width,
        height = plot_height,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        yaxis_title = f'RSI (%)',
        title = dict(
            text = title_rsi,
            font_size = title_font_size,
            y = 0.95,
            x = 0.45,
            xanchor = 'center',
            yanchor = 'top'
        )
    )
    fig_rsi.update_xaxes(
        type = 'category',
        nticks = n_ticks_max,
        tickangle = -90,
        ticks = 'outside',
        ticklen = 8,
        ticklabelshift = 5,  # not working
        ticklabelstandoff = 10,  # not working
    )
    fig_rsi.update_yaxes(
        secondary_y = False,
        range = (0, 100),
        nticks = 11,
        ticks = 'outside',
        ticklen = 8,
        ticklabelshift = 5,  # not working
        ticklabelstandoff = 10,  # not working
    )
    if overlay_price:
        fig_rsi.update_yaxes(
            title_text = price_name,
            secondary_y = True,
            ticks = 'outside',
            ticklen = 8,
            ticklabelshift = 5,  # not working
            ticklabelstandoff = 10,  # not working
            showgrid = False
        )

    return fig_rsi


In [5]:
# theme = 'light'
theme = 'dark'

# x_min = datetime(2023, 12, 20)
# x_max = datetime(2024, 9, 21)

# rsi = relative_strength(adj_close_tk[x_min: x_max])
rsi_data = relative_strength(adj_close_tk)

fig_rsi = plot_rsi_hlines_plotly(
    rsi_data,
    tk,
    # df_price = adj_close_tk[x_min: x_max],
    df_price = adj_close_tk,
    theme = theme,
    overlay_price = True,
    price_type = 'adj close'
)
fig_rsi.show()

Stochastic Oscillator

In [6]:
def stochastic_oscillator(
    close_tk,
    high_tk,
    low_tk,
    fast_window = 14,
    slow_window = 3
):
    
    fast_low = low_tk.rolling(window = fast_window, min_periods = 1).min()
    fast_high = high_tk.rolling(window = fast_window, min_periods = 1).max()
    k_line = 100 * (close_tk - fast_low) / (fast_high - fast_low)

    d_line = k_line.rolling(window = slow_window, min_periods = 1).mean()

    k_line.index = k_line.index.astype(str)
    d_line.index = d_line.index.astype(str)
    stochastic_type = f'({fast_window}, {slow_window})'

    stochastic_data = {
        'k_line': k_line,
        'd_line': d_line,
        'type': stochastic_type
    }

    return stochastic_data

    # return stochastic_data

In [7]:
# df_stochastic = stochastic_oscillator(close_tk, high_tk, low_tk)
# display(df_stochastic)
# df_stochastic.to_csv('../output/stochastic_test.csv')

In [8]:
def plot_stochastic_plotly(
    stochastic_data,
    tk,
    oversold_threshold = 20,
    overbought_threshold = 80,
    n_ticks_max = 48,
    plot_width = 1450,
    plot_height = 750,
    title_font_size = 32,
    theme = 'dark',
    overlay_price = False,
    prices = None
):
    """
    stochastic_data: output from stochastic_oscillator()
    tk: ticker for which to plot the stochastic %K and %D lines
    prices: close_tk (if overlay_price is True)

    """

    k_line = stochastic_data['k_line']
    d_line = stochastic_data['d_line']
    stochastic_type = stochastic_data['type']
    
    style = theme_style[theme]

    title_stochastic = f'{tk} {stochastic_type} Stochastic Oscillator (%)'
        
    if overlay_price:
        price_name = 'Close'
        prices.index = prices.index.astype(str)
        fig_stochastic = make_subplots(specs=[[{'secondary_y': True}]])
    else:
        fig_stochastic = make_subplots(rows = 1, cols = 1)

    x_min = k_line.index.min()
    x_max = k_line.index.max()
    
    stochastic_hlines = pd.DataFrame(
        {
            'oversold': oversold_threshold,
            'overbought': overbought_threshold,
            '100': 100
        },
        index = k_line.index
    )

    # For some reason, the price overlay trace shows first in the legend if it's added last
    if overlay_price:
        fig_stochastic.add_trace(
            go.Scatter(
                x = prices.index,
                y = prices,
                # y = close_tk,
                line_color = style['basecolor'],
                name = price_name
            ),
            secondary_y = True
        )
    fig_stochastic.add_trace(
        go.Scatter(
            x = stochastic_hlines.index,
            y = stochastic_hlines['oversold'],
            line_color = style['oversold_linecolor'],
            line_width = 2,
            fill = 'tozeroy',
            fillcolor = style['oversold_fillcolor'],
            name = f'Oversold < {oversold_threshold}%'
        ),
        secondary_y = False
    )
    fig_stochastic.add_trace(
        go.Scatter(
            x = stochastic_hlines.index,
            y = stochastic_hlines['100'],
            line_color = 'black',
            line_width = 0,
            showlegend = False
        ),
        secondary_y = False
    )
    fig_stochastic.add_trace(
        go.Scatter(
            x = stochastic_hlines.index,
            y = stochastic_hlines['overbought'],
            line_color = style['overbought_linecolor'],
            line_width = 2,
            fill = 'tonexty',  # fill to previous scatter trace
            fillcolor = style['overbought_fillcolor'],
            name = f'Overbought > {overbought_threshold}%'
        ),
        secondary_y = False
    )
    fig_stochastic.add_trace(
        go.Scatter(
            x = d_line.index,
            y = d_line,
            line_color = style['dline_linecolor'],
            line_width = 2,        
            name = f'{stochastic_type} %D Line'
        ),
        secondary_y = False
    )
    fig_stochastic.add_trace(
        go.Scatter(
            x = k_line.index,
            y = k_line,
            line_color = style['kline_linecolor'],
            line_width = 2,        
            name = f'{stochastic_type} %K Line'
        ),
        secondary_y = False
    )

    # Add plot border
    fig_stochastic.add_shape(
        type = 'rect',
        xref = 'x',  # use 'x' because of seconday axis - 'paper' does not work correctly
        yref = 'paper',
        x0 = x_min,
        x1 = x_max,
        y0 = 0,
        y1 = 1,
        line_color = style['x_linecolor'],
        line_width = 0.3
    )
    
    # Update layout and axes
    fig_stochastic.update_layout(
        width = plot_width,
        height = plot_height,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        yaxis_title = f'Stochastic Oscillator (%)',
        title = dict(
            text = title_stochastic,
            font_size = title_font_size,
            y = 0.95,
            x = 0.45,
            xanchor = 'center',
            yanchor = 'top'
        )
    )
    fig_stochastic.update_xaxes(
        type = 'category',
        gridcolor = style['x_gridcolor'],
        showgrid = True,
        nticks = n_ticks_max,
        tickangle = -90,
        ticks = 'outside',
        ticklen = 8,
        ticklabelshift = 5,  # not working
        ticklabelstandoff = 10  # not working
    )
    fig_stochastic.update_yaxes(
        secondary_y = False,
        range = (0, 100),
        gridcolor = style['y_gridcolor'],
        showgrid = True,
        nticks = 11,
        ticks = 'outside',
        ticklen = 8,
        ticklabelshift = 5,  # not working
        ticklabelstandoff = 10  # not working
    )
    if overlay_price:
        fig_stochastic.update_yaxes(
            title_text = price_name,
            secondary_y = True,
            ticks = 'outside',
            ticklen = 8,
            ticklabelshift = 5,  # not working
            ticklabelstandoff = 10,  # not working
            showgrid = False
        )

    return fig_stochastic


In [11]:
theme = 'light'
theme = 'dark'

x_min = datetime(2024, 3, 21)
x_max = datetime(2024, 9, 21)

stochastic_data = stochastic_oscillator(close_tk[x_min: x_max], high_tk[x_min: x_max], low_tk[x_min: x_max])
fig_stochastic = plot_stochastic_plotly(stochastic_data, tk, theme = theme, overlay_price = True, prices = close_tk[x_min: x_max])
# fig_stochastic = plot_stochastic_plotly(k_line, d_line, tk, theme = theme)
fig_stochastic.show()